## AISE4010- Assignment3 - Time Series Classification using TCN and Transformer + Hyperparameter Tuning

## Grade: 100 points

### Instructions

#### Follow These Steps before submitting your assignment

1. Complete the notebook.

2. Make sure all plots have axis labels.

3. Once the notebook is complete, `Restart` your kernel by clicking 'Kernel' > 'Restart & Run All'.

4. Fix any errors until your notebook runs without any problems.

5. Submit one completed notebook for the group to OWL by the deadline.

6. Make sure to reference all external code and documentation used.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import (Conv1D, GlobalAveragePooling1D, Dense, 
                                      Dropout, Flatten, MultiHeadAttention, 
                                      LayerNormalization, Activation)
from tensorflow.keras.optimizers import Adam
from keras_tuner import GridSearch
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras_tuner'

### Dataset

The dataset is a sample of 46 satellite images, collected in 2006, located in southwestern France near Toulouse. It
is a 24 km × 24 km area and the dataset uses 3 output classes (2 available) for arable soil classification based on the following paper: https://arxiv.org/pdf/1811.10166.

You will be using helper functions below to prepare it for deep learning models.

In [ ]:
# Call this helper method by passing in the names of the provided training and test sets' files.
def read_SITS_data(name_file):
    data = pd.read_table(name_file, sep=',', header=None)

    y_data = data.iloc[:,0]
    y = np.asarray(y_data.values, dtype='uint8')
    y[y>1] = 0

    polygonID_data = data.iloc[:,1]
    polygon_ids = polygonID_data.values
    polygon_ids = np.asarray(polygon_ids, dtype='uint16')

    X_data = data.iloc[:,2:]
    X = X_data.values
    X = np.asarray(X, dtype='float32')

    return  X, polygon_ids, y

In [ ]:
def custom_feature_scaling(train, test):
    min_per = np.percentile(train, 2, axis=(0,1))
    max_per = np.percentile(train, 100-2, axis=(0,1))

    new_train = (train-min_per)/(max_per-min_per)
    new_test = (test-min_per)/(max_per-min_per)

    return new_train, new_test

### Question 1 - Data Preprocessing (15%)
- Q1.1 Call "read_SITS_data()" for the training set and store the results as X_train, polygon_ids_train, and y_train.
- Q1.2 Call "read_SITS_data()" above for the test set and store the results as X_test, polygon_ids_test, and y_test.
- Q1.3 Reshape the training and test sets.
  - Each set must be reshaped into a 3-D array. The first dimension will be the number of rows of the original set. The second dimension will be int(x / 3), where x is the number of columns of the original set and int() is a casting function. The third dimension will be 3 (number of channels).
- Q1.4 Call "custom_feature_scaling()" with the training and test sets. Save the results as the final sets for use.
- Q1.5 How many entries are in the training set? How many time steps are in each entry? How many features are there for each time step? How many labels for each entry?


In [ ]:
# Q1.1 - Load training data
X_train, polygon_ids_train, y_train = read_SITS_data('train_dataset.csv')

# Q1.2 - Load test data
X_test, polygon_ids_test, y_test = read_SITS_data('test_dataset.csv')

print(f"X_train shape before reshape: {X_train.shape}")
print(f"X_test shape before reshape: {X_test.shape}")

# Q1.3 - Reshape data into 3D arrays
# First dimension: number of rows
# Second dimension: int(columns / 3)
# Third dimension: 3 (number of channels)
n_train = X_train.shape[0]
n_test = X_test.shape[0]
n_features = X_train.shape[1]
time_steps = int(n_features / 3)

X_train = X_train.reshape(n_train, time_steps, 3)
X_test = X_test.reshape(n_test, time_steps, 3)

print(f"X_train shape after reshape: {X_train.shape}")
print(f"X_test shape after reshape: {X_test.shape}")

# Q1.4 - Apply custom feature scaling
X_train, X_test = custom_feature_scaling(X_train, X_test)

print(f"\nFinal X_train shape: {X_train.shape}")
print(f"Final X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

*Write your Answer to Q1.5 here:*

**Q1.5 Answer:**
- **Number of entries in the training set:** 46 (the number of samples/satellite images)
- **Number of time steps in each entry:** 24 (derived from 72 columns / 3 = 24 time steps)
- **Number of features for each time step:** 3 (the three channels representing different spectral bands)
- **Number of labels for each entry:** 1 (binary classification label per entry)

### Question2 - Temporal Convolutional Network
- Q2.1 Create a Sequential model for classification. The model should have a TCN layer of size 64, a fully connected layer of size 256, a dropout of 0.3, and a fully connected output layer with Softmax activation (Hint: the logits axis should be on 0). Train the model using the provided dataset for 20 epochs. Use the batch_size of 32, and ADAM optimizer. Print the model summary.
- Q2.2 Train the model with the same parameters, print the model summary and evaluate the model's accuracy on the test set. Print the accuracy.
- Q2.3 Why do we use the Softmax activation on the output layer? In what scenarios does this contrast to using ReLU instead?


In [ ]:
# Q2.1 and Q2.2 - Create and train TCN model
from tensorflow.keras.layers import TCN

# Create Sequential model with TCN
model_tcn = Sequential([
    TCN(64, kernel_size=5, dilations=[1, 2, 4, 8], input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(256),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

# Print model summary
print("=" * 60)
print("TCN Model Summary:")
print("=" * 60)
model_tcn.summary()

# Compile and train the model
model_tcn.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTraining TCN Model...")
history_tcn = model_tcn.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    verbose=1
)

# Evaluate on test set
print("\nEvaluating TCN Model on Test Set...")
test_loss, test_accuracy = model_tcn.evaluate(X_test, y_test, verbose=0)
print(f"TCN Test Accuracy: {test_accuracy:.4f}")

*Write your Answer to Q2.3 Here:*

**Q2.3 Answer:**

**Why Softmax on Output Layer:**
Softmax activation is used on the output layer for multi-class classification problems because it:
1. Converts raw logits into probability distributions that sum to 1
2. Ensures each output neuron represents the probability of a specific class
3. Makes it suitable for use with categorical/sparse_categorical_crossentropy loss functions
4. Handles multiple mutually exclusive classes where only one label should be true

**Contrast with ReLU:**
- **ReLU** (Rectified Linear Unit) outputs unbounded positive values (max(0, x)), which don't naturally form probabilities and can sum to any positive value
- **ReLU** is inappropriate for classification output layers as it doesn't constrain outputs to probability ranges [0,1]
- **ReLU** is ideal for hidden layers to introduce non-linearity and allow the network to learn complex patterns
- Using ReLU on output would require additional normalization to interpret results as probabilities

In summary, Softmax ensures valid probability distributions for classification, while ReLU is better suited for hidden layer activations in deep networks.

### Question 3 - Transformer Model
- Q3.1 Create a transformer encoder block. It should use MultiHeadAttention for residual connection. The projection layers can be two Conv1D layers, based on number of feed forward dimensions and with kernel sizes of 1.
- Q3.2 Define the model. It should have 4 encoder blocks, each with 256 heads and feed forward dimensions of 4. Add a flatten layer, then a fully connected layer of size 2 and a fully connected output layer.
- Q3.3 Print the model summary, train the model using 50 epochs and a batch size of 32. Evaluate the model accuracy on the test set and print it.


In [ ]:
# Q3.1 - Create Transformer encoder block with MultiHeadAttention
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """
    Transformer encoder block with MultiHeadAttention and residual connection
    
    Args:
        inputs: input tensor
        head_size: size of each attention head
        num_heads: number of attention heads
        ff_dim: feed forward dimensions
        dropout: dropout rate
    """
    # Multi-head attention with residual connection
    x = MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=head_size, 
        dropout=dropout
    )(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + inputs)
    
    # Feed forward network with residual connection
    x_res = x
    x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x + x_res)
    
    return x

# Q3.2 - Define the complete Transformer model
def create_transformer_model(input_shape, num_transformer_blocks=4, head_size=256, 
                             num_heads=4, ff_dim=4, dropout=0.1):
    """
    Create a transformer model for time series classification
    """
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
    # Apply 4 transformer encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim * input_shape[-1], dropout)
    
    # Flatten and fully connected layers
    x = Flatten()(x)
    x = Dense(2)(x)
    outputs = Dense(2, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Create the model
model_transformer = create_transformer_model(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    num_transformer_blocks=4,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    dropout=0.1
)

# Q3.3 - Print model summary, train, and evaluate
print("=" * 60)
print("Transformer Model Summary:")
print("=" * 60)
model_transformer.summary()

# Compile the model
model_transformer.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nTraining Transformer Model...")
history_transformer = model_transformer.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    verbose=1
)

# Evaluate on test set
print("\nEvaluating Transformer Model on Test Set...")
test_loss_trans, test_accuracy_trans = model_transformer.evaluate(X_test, y_test, verbose=0)
print(f"Transformer Test Accuracy: {test_accuracy_trans:.4f}")

### Question 4 - Hyperparameter Tuning
- Q4.1 Define a search space for the number of neurons in the fully connected layer that follows the flatten layer. The lower bound should be 2, the upper bound should be 16, and it should search every other value in between. Also have the tuner decide whether or not a dropout layer of 0.3 should be added after the aforementioned layer.
- Q4.2 Using GridSearch, search for the best hyperparameters with respect to accuracy over 50 epochs.
- Q4.3 Using the best hyperparameters, rebuild the model and print the model accuracy.

In [ ]:
# Q4.1 - Define the model builder function with hyperparameter tuning
def build_model(hp):
    """
    Build a transformer model with tunable hyperparameters
    """
    # Define search space for number of neurons in fully connected layer
    # Lower bound: 2, Upper bound: 16, search every other value (step=2)
    fc_units = hp.Int('fc_units', min_value=2, max_value=16, step=2)
    
    # Define whether to add dropout layer
    use_dropout = hp.Boolean('use_dropout')
    
    # Build model
    inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = inputs
    
    # Transformer encoder blocks
    for _ in range(2):
        x = transformer_encoder(x, 128, 2, 4 * X_train.shape[2], dropout=0.1)
    
    # Flatten
    x = Flatten()(x)
    
    # Fully connected layer with tunable units
    x = Dense(fc_units)(x)
    
    # Optional dropout
    if use_dropout:
        x = Dropout(0.3)(x)
    
    # Output layer
    outputs = Dense(2, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile
    model.compile(
        optimizer=Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Q4.2 - Use GridSearch to find best hyperparameters
print("=" * 60)
print("Starting GridSearch for Hyperparameter Tuning...")
print("=" * 60)

tuner = GridSearch(
    build_model,
    objective='accuracy',
    max_trials=24,  # 2 options for dropout × 12 values for fc_units (2,4,6,8,10,12,14,16)
    directory='grid_search_results',
    project_name='transformer_tuning'
)

print("\nRunning GridSearch...")
tuner.search(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    verbose=0
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"\nBest Hyperparameters Found:")
print(f"  FC Units: {best_hps.get('fc_units')}")
print(f"  Use Dropout: {best_hps.get('use_dropout')}")

# Q4.3 - Rebuild model with best hyperparameters and evaluate
print("\n" + "=" * 60)
print("Building Final Model with Best Hyperparameters...")
print("=" * 60)

best_model = tuner.hypermodel.build(best_hps)
print("\nBest Model Summary:")
best_model.summary()

print("\nTraining Best Model...")
best_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    verbose=0
)

# Evaluate on test set
print("\nEvaluating Best Model on Test Set...")
best_test_loss, best_test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model Test Accuracy: {best_test_accuracy:.4f}")

### Question 6 - Discussion (5%)
- Q6.1 Indicate other hyperparameters relevant to transformers that can be tuned.
- Q6.2 What are the advantages and disadvantages of using GridSearch for finding optimal hyperparameters?


*Write your Answer to Q6.1 and Q6.2 Here:*

**Q6.1 - Other Hyperparameters Relevant to Transformers:**

Key transformer hyperparameters that can be tuned include:

1. **Number of Attention Heads:** Controls the dimensionality of attention mechanisms. More heads capture different aspects of relationships in the data.

2. **Embedding Dimension (d_model):** The size of the feature vectors used throughout the transformer. Larger dimensions can capture more information but require more computation.

3. **Feed Forward Dimension (ff_dim):** The hidden layer size in the feed-forward networks within encoder blocks. Typically 2-4 times the embedding dimension.

4. **Number of Encoder Layers:** Stacking more transformer blocks increases model capacity and depth but also increases computational cost.

5. **Dropout Rate:** Controls regularization strength to prevent overfitting. Common values range from 0.1 to 0.5.

6. **Learning Rate:** Controls the step size during optimization. Affects convergence speed and final model performance.

7. **Batch Size:** Impacts training stability and memory usage.

8. **Head Size:** The dimensionality of each individual attention head (d_model / num_heads).

9. **Activation Functions:** Choice of activation functions in feed-forward networks (ReLU, GELU, etc.).

10. **Layer Normalization Position:** Pre-normalization vs post-normalization configuration.

---

**Q6.2 - Advantages and Disadvantages of GridSearch:**

**Advantages:**
- **Exhaustive Search:** Guarantees finding the best combination within the defined search space
- **Simplicity:** Easy to understand and implement
- **Reproducibility:** Deterministic results across runs
- **Interpretability:** Can easily see how each hyperparameter affects performance
- **Parallelizable:** Can run multiple trials in parallel on distributed systems
- **No Assumptions:** Doesn't assume any functional relationship between hyperparameters and performance

**Disadvantages:**
- **Computational Expense:** Exponentially increases with number of hyperparameters (curse of dimensionality)
  - Example: 3 hyperparameters with 10 values each = 1,000 trials
- **Inefficient Search:** Tests uninformative combinations, wasting computational resources
- **Scalability Issues:** Impractical for large search spaces or many hyperparameters
- **Time-Consuming:** Can take hours or days for large grids with complex models
- **Limited Scope:** Only searches discrete predefined values, missing optimal continuous values between them
- **No Learning:** Doesn't use information from previous trials to inform the next search
- **Coarse Granularity:** Resolution depends on number of predefined values

**Better Alternatives:**
- **Random Search:** More efficient for large search spaces
- **Bayesian Optimization:** Uses probabilistic models to guide search
- **Hyperband/Successive Halving:** Progressively eliminates poor performing configurations